In [ ]:
import sqlite3

conn = sqlite3.connect('database.db')
c = conn.cursor()

def create_tables():
    c.execute('''
    CREATE TABLE IF NOT EXISTS Usuarios (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT NOT NULL UNIQUE,
        email TEXT NOT NULL,
        password TEXT NOT NULL
    )
    ''')
    c.execute('''
    CREATE TABLE IF NOT EXISTS Extrato (
        ID_Transacao INTEGER PRIMARY KEY AUTOINCREMENT,
        ID_User INTEGER,
        Data_da_Transacao DATE,
        Transacao TEXT,
        Tipo_Transacao TEXT,
        Identificacao TEXT,
        Valor REAL,
        Entrada REAL,
        Saida REAL,
        Mes TEXT,
        Saldo REAL
    )
    ''')
    conn.commit()

create_tables()

In [3]:
import sqlite3

# Criar conexão com o banco de dados
conn = sqlite3.connect('database.db')
c = conn.cursor()

# Verificar campos da tabela Usuarios
c.execute("PRAGMA table_info(Usuarios)")
print("Campos da tabela Usuarios:")
for row in c.fetchall():
    print(f"  {row[1]} ({row[2]})")

# Verificar registros da tabela Usuarios
c.execute("SELECT * FROM Usuarios")
print("\nRegistros da tabela Usuarios:")
for row in c.fetchall():
    print(f"  {row}")

# Verificar campos da tabela Jogos
c.execute("PRAGMA table_info(Extrato)")
print("\nCampos da tabela Extrato:")
for row in c.fetchall():
    print(f"  {row[1]} ({row[2]})")

# Verificar registros da tabela Jogos
c.execute("SELECT * FROM Extrato")
print("\nRegistros da tabela Extrato:")
for row in c.fetchall():
    print(f"  {row}")

# Fechar a conexão
conn.close()

Campos da tabela Usuarios:
  id (INTEGER)
  username (TEXT)
  password (TEXT)

Registros da tabela Usuarios:

Campos da tabela Extrato:
  ID_Transacao (INTEGER)
  ID_User (INTEGER)
  Data_da_Transacao (DATE)
  Transacao (TEXT)
  Tipo_Transacao (TEXT)
  Identificacao (TEXT)
  Valor (REAL)
  Entrada (REAL)
  Saida (REAL)
  Mes (TEXT)
  Saldo (REAL)

Registros da tabela Extrato:
  (1, 1, '31/01/2024', 'Transf Pix recebida', 'CRÉDITO', 'JESSICA BOMFIM OLIVEIRA', 5.0, 5.0, 0.0, 'Janeiro', 5.0)
  (2, 1, '31/01/2024', 'Transf Pix recebida', 'CRÉDITO', 'ADEILTON PRAZERES BARBOSA', 100.0, 100.0, 0.0, 'Janeiro', 100.0)
  (3, 1, '31/01/2024', 'Transf Pix recebida', 'CRÉDITO', 'Elias dos Santos santana', 100.0, 100.0, 0.0, 'Janeiro', 100.0)
  (4, 1, '31/01/2024', 'Transf Pix recebida', 'CRÉDITO', 'MARIO HENRIQUE FERREIRA MIRANDA', 160.0, 160.0, 0.0, 'Janeiro', 160.0)
  (5, 1, '31/01/2024', 'Transf Pix recebida', 'CRÉDITO', 'GEORGE CLEBER MORAES LIMA', 100.0, 100.0, 0.0, 'Janeiro', 100.0)
  (6, 1, 